### 1 - Import test data

In [7]:
import numpy as np
import pandas as pd

df = pd.DataFrame({
    "value": [100, 110, 120, 130, 140, 90, 91, 28]})

nguong = 5


###2- NUMPY CỰC DỄ HIỂU (VÒNG FOR – KHÔNG MA TRẬN)

In [ ]:

nguong = 5
value = df["value"].to_numpy() # convert pandas Series to numpy array

n = len(value)

check = np.zeros(n, dtype=int) # Tạo 1 mảng NumPy có độ dài = n,tất cả phần tử = 0,kiểu số nguyên
print(check)
for i in range(n):

    # check = 3
    if i >= 3:
        if (
            value[i]   < value[i-3] - nguong and
            value[i-1] < value[i-3] - nguong and
            value[i-2] < value[i-3] - nguong
        ):
            check[i] = 3
            continue

    # check = 2
    if i >= 2:
        if (
            value[i]   < value[i-2] - nguong and
            value[i-1] < value[i-2] - nguong
        ):
            check[i] = 2
            continue

    # check = 1
    if i >= 1:
        if value[i] < value[i-1] - nguong:
            check[i] = 1

df["check"] = check
print(df)


[0 0 0 0 0 0 0 0]
   value  check
0    100      0
1    110      0
2    120      0
3    130      0
4    140      0
5     90      1
6     91      2
7     28      3


###3 VECTORIZE

In [11]:

nguong = 5
v = df["value"].to_numpy()  # convert pandas Series to numpy array    
n = len(v)

# =============================
# INIT CHECK ARRAY
# =============================
check_ = np.zeros(n, dtype=int)

# =============================
# CHECK = 1
# t0 < t1 - nguong
# =============================
cond1 = v[1:] < (v[:-1] - nguong)
check_[1:][cond1] = 1

# =============================
# CHECK = 2
# t0 < t2 - nguong AND t1 < t2 - nguong
# =============================
cond2 = (
    (v[2:]   < v[:-2] - nguong) &
    (v[1:-1] < v[:-2] - nguong)
)
check_[2:][cond2] = 2

# =============================
# CHECK = 3
# t0 < t3 - nguong AND t1 < t3 - nguong AND t2 < t3 - nguong
# =============================
cond3 = (
    (v[3:]   < v[:-3] - nguong) &
    (v[2:-1] < v[:-3] - nguong) &
    (v[1:-2] < v[:-3] - nguong)
)
check_[3:][cond3] = 3

# =============================
# OUTPUT
# =============================
df["check_"] = check
print(df)


   value  check  check_
0    100      0       0
1    110      0       0
2    120      0       0
3    130      0       0
4    140      0       0
5     90      1       1
6     91      2       2
7     28      3       3


##4 (vectorized + loop theo N, KHÔNG loop theo i)

✅ PHIÊN BẢN 1 – DỄ HIỂU, ĐÚNG TƯ DUY
🎯 Ý tưởng

Không loop theo dữ liệu

Chỉ loop theo N (rất nhỏ: 1–5–10)

Mỗi N tạo 1 condN

Nếu đúng → gán check = N (ghi đè → ưu tiên N lớn)

In [ ]:

def compute_check(value, nguong, max_N):
    """
    value  : numpy array 1D
    nguong : threshold
    max_N  : số mẫu tối đa (vd: 3)
    """
    v = value
    n = len(v)

    check = np.zeros(n, dtype=int) # set    all to 0

    for N in range(1, max_N + 1):
        
        cond = np.ones(n - N, dtype=bool) # tạo mask ban đầu
        print(f"N={N}, initial cond: {cond}")
        # so sánh t0..t(N-1) với mốc tN
        for k in range(N):
            cond &= (v[N-k : n-k] < v[: n-N] - nguong)
            print(f"  k={k}, cond: {cond}")
        check[N:][cond] = N # gán kết quả (ưu tiên N lớn)
        print(f"  after setting check for N={N}: {check}")

    return check

df["check_1"] = compute_check(df["value"].to_numpy(), nguong=5 ,max_N=3)
print(df)


N=1, initial cond: [ True  True  True  True  True  True  True]
  k=0, cond: [False False False False  True False  True]
  after setting check for N=1: [0 0 0 0 0 1 0 1]
N=2, initial cond: [ True  True  True  True  True  True]
  k=0, cond: [False False False  True  True  True]
  k=1, cond: [False False False False  True False]
  after setting check for N=2: [0 0 0 0 0 1 2 1]
N=3, initial cond: [ True  True  True  True  True]
  k=0, cond: [False False  True  True  True]
  k=1, cond: [False False False  True  True]
  k=2, cond: [False False False False  True]
  after setting check for N=3: [0 0 0 0 0 1 2 3]
   value  check  check_  check_1
0    100      0       0        0
1    110      0       0        0
2    120      0       0        0
3    130      0       0        0
4    140      0       0        0
5     90      1       1        1
6     91      2       2        2
7     28      3       3        3


### check nguong fix

In [27]:
nguongFix = 88
ArryNguongFix = df["value"].to_numpy()  # convert pandas Series to numpy array  
checkNguongfix = np.zeros(len(ArryNguongFix), dtype=int)
checkNguongfix = ArryNguongFix < nguongFix # tạo mask True / False
df["checkNguongfix"] = checkNguongfix
# thêm cột mô tả
# df["note"] = np.where(checkNguongfix,f"thấp hơn ngưỡng co dinh {nguongFix}",f"cao hơn ngưỡng co dinh {nguongFix}")
df["note"] = (
    np.where(df["checkNguongfix"],
             f"thấp hơn ngưỡng cố định {nguongFix}",
             f"cao hơn ngưỡng cố định {nguongFix}")
    +
    np.where(df["check"] > 0,
             " | vi phạm " + df["check"].astype(str) + " mẫu",
             "")
)

print(df)


   value  check  check_  check_1  checkNguongfix  \
0    100      0       0        0           False   
1    110      0       0        0           False   
2    120      0       0        0           False   
3    130      0       0        0           False   
4    140      0       0        0           False   
5     90      1       1        1           False   
6     91      2       2        2           False   
7     28      3       3        3            True   

                                         note  
0                   cao hơn ngưỡng cố định 88  
1                   cao hơn ngưỡng cố định 88  
2                   cao hơn ngưỡng cố định 88  
3                   cao hơn ngưỡng cố định 88  
4                   cao hơn ngưỡng cố định 88  
5   cao hơn ngưỡng cố định 88 | vi phạm 1 mẫu  
6   cao hơn ngưỡng cố định 88 | vi phạm 2 mẫu  
7  thấp hơn ngưỡng cố định 88 | vi phạm 3 mẫu  
